In [13]:
# !pip install deepface opencv-python gradio scikit-image dlib

import gradio as gr
import cv2
import numpy as np
import dlib
from PIL import Image
from deepface import DeepFace
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive
drive.mount('/content/drive')

# OpenCV 얼굴 검출기 초기화
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# 가중치 설정
weights = {
    "feature_distribution": 0.2,  # 이목구비의 분포
    "eye_color": 0.05,            # 눈동자 색
    "nose_shape": 0.15,            # 코의 모양
    "mouth_shape": 0.15,           # 입의 모양
    "eye_position_ratio": 0.15,    # 눈 위치 비율
    "nose_position_ratio": 0.15,   # 코 위치 비율
    "mouth_position_ratio": 0.15  # 입 위치 비율
}
# 얼굴 검출 함수
def detect_face_opencv(image):
    image_np = np.array(image.convert("RGB"))
    gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return len(faces) > 0

# 얼굴 인식 상태 확인 함수
def check_face(image, role):
    if not detect_face_opencv(image):
        return f"{role}: 얼굴 인식이 불가합니다.", False
    return f"{role}: 얼굴 인식 성공!", True

# 얼굴 인식 결과 취합
def update_recognition_status(father_img, mother_img, child_img, sibling_files):
    roles = ["Father", "Mother", "Child"]
    images = [father_img, mother_img, child_img]
    sibling_images = [Image.open(f) for f in sibling_files] if sibling_files else []

    all_images = images + sibling_images
    all_roles = roles + [f"Sibling {i+1}" for i in range(len(sibling_images))]
    messages = []
    success = True

    for img, role in zip(all_images, all_roles):
        if img is not None:
            message, detected = check_face(img, role)
            messages.append(message)
            if not detected:
                success = False

    return "\n".join(messages), success

# 코사인 유사도 계산 함수
def calculate_cosine_similarity(embedding1, embedding2):
    embedding1 = np.array(embedding1).reshape(1, -1)
    embedding2 = np.array(embedding2).reshape(1, -1)
    return cosine_similarity(embedding1, embedding2)[0][0]

# RGB 유사도 계산 함수
def calculate_rgb_similarity(img1, img2):
    """
    두 이미지의 평균 RGB 유사도를 계산합니다.
    Args:
        img1, img2: numpy 배열 (RGB 형식)
    Returns:
        float: RGB 유사도 (0 ~ 1)
    """
    # 이미지를 RGB로 변환
    if img1.shape[-1] != 3 or img2.shape[-1] != 3:
        raise ValueError("이미지가 RGB 형식이 아닙니다.")

    # 이미지의 평균 RGB 값 계산
    avg_rgb1 = np.mean(img1, axis=(0, 1))
    avg_rgb2 = np.mean(img2, axis=(0, 1))

    # RGB 유사도 계산
    diff = np.linalg.norm(avg_rgb1 - avg_rgb2)
    return 1 - (diff / np.sqrt(3 * (255 ** 2)))  # 0~1 범위로 정규화


# SSIM 유사도 계산
def calculate_ssim_similarity(image1, image2, target_size=(224, 224)):
    """
    두 이미지 간 구조적 유사도 (SSIM)를 계산합니다.
    Args:
        image1, image2: numpy 배열 (BGR 형식)
        target_size: SSIM 계산을 위한 리사이즈 크기
    Returns:
        float: SSIM 값
    """
    try:
        # 이미지를 동일한 크기로 리사이즈
        resized_img1 = cv2.resize(image1, target_size, interpolation=cv2.INTER_AREA)
        resized_img2 = cv2.resize(image2, target_size, interpolation=cv2.INTER_AREA)

        # 흑백 변환
        gray1 = cv2.cvtColor(resized_img1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(resized_img2, cv2.COLOR_BGR2GRAY)

        # SSIM 계산
        return ssim(gray1, gray2)
    except Exception as e:
        raise RuntimeError(f"SSIM 계산 중 오류 발생: {str(e)}")

def landmarks_to_numpy(landmarks):
    """
    Dlib 랜드마크 객체를 NumPy 배열로 변환합니다.

    Args:
        landmarks: Dlib의 full_object_detection 객체

    Returns:
        np.ndarray: 랜드마크 좌표 배열 (N x 2)
    """
    if not isinstance(landmarks, dlib.full_object_detection):
        raise TypeError("Input must be a dlib.full_object_detection object.")

    num_points = landmarks.num_parts
    points_array = np.array([[landmarks.part(i).x, landmarks.part(i).y] for i in range(num_points)])
    return points_array

def calculate_lip_curvature_similarity(landmarks1, landmarks2):
    """
    두 얼굴의 입술 랜드마크 기반 곡률 유사도를 계산합니다.

    Args:
        landmarks1: 첫 번째 얼굴의 랜드마크 배열 (numpy 배열)
        landmarks2: 두 번째 얼굴의 랜드마크 배열 (numpy 배열)

    Returns:
        float: 입술 곡률 유사도 (0 ~ 1 범위)
    """
    # 입력 데이터 검증
    if landmarks1.shape[0] < 60 or landmarks2.shape[0] < 60:
        raise ValueError("Landmark arrays must have at least 60 points.")

    # 입술 랜드마크 좌표 (48~59번 점)
    top_curve_diff = np.linalg.norm(landmarks1[48:54] - landmarks2[48:54], axis=1).mean()
    bottom_curve_diff = np.linalg.norm(landmarks1[54:60] - landmarks2[54:60], axis=1).mean()

    # 정규화 (최대 좌표값 기준)
    max_possible_diff = 1000  # 예상 가능한 최대 좌표 차이
    normalized_top = top_curve_diff / max_possible_diff
    normalized_bottom = bottom_curve_diff / max_possible_diff

    # 유사도 계산 (0 ~ 1 범위)
    similarity = 1 - (normalized_top + normalized_bottom) / 2

    # 비정상 값 처리
    similarity = max(0, min(similarity, 1))  # 유사도를 0~1로 제한
    return similarity

def calculate_position_ratio_similarity(landmarks1, landmarks2, position_index, max_diff=0.2):
    """
    랜드마크를 기반으로 특정 부위의 위치 비율 유사도를 계산합니다.

    Args:
        landmarks1 (numpy.ndarray): 첫 번째 얼굴의 랜드마크 배열.
        landmarks2 (numpy.ndarray): 두 번째 얼굴의 랜드마크 배열.
        position_index (int): 비교할 랜드마크 포인트의 인덱스.
        max_diff (float): 최대 허용 비율 차이 (기본값: 0.2).

    Returns:
        float: 0~1 범위의 유사도 점수.
    """
    def position_ratio(landmarks, position_index):
        forehead = landmarks[27]  # 이마 중앙점
        chin = landmarks[8]  # 턱 중앙점
        face_length = np.linalg.norm(forehead - chin)
        if face_length == 0:
            raise ValueError("Invalid face length: forehead and chin landmarks are identical.")
        position = landmarks[position_index]
        return (position[1] - forehead[1]) / face_length

    ratio1 = position_ratio(landmarks1, position_index)
    ratio2 = position_ratio(landmarks2, position_index)
    diff = abs(ratio1 - ratio2)
    similarity = 1 - (diff / max_diff)
    return max(0, min(similarity, 1))


def calculate_eye_position_ratio_similarity(landmarks1, landmarks2, max_diff=0.2):
    """
    두 얼굴의 눈 위치 비율 유사도를 계산합니다.

    Args:
        landmarks1 (numpy.ndarray): 첫 번째 얼굴의 랜드마크 배열.
        landmarks2 (numpy.ndarray): 두 번째 얼굴의 랜드마크 배열.
        max_diff (float): 최대 허용 비율 차이 (기본값: 0.2).

    Returns:
        float: 0~1 범위의 유사도 점수.
    """
    return calculate_position_ratio_similarity(landmarks1, landmarks2, position_index=36, max_diff=max_diff)


def calculate_nose_position_ratio_similarity(landmarks1, landmarks2, max_diff=0.2):
    """
    두 얼굴의 코 위치 비율 유사도를 계산합니다.

    Args:
        landmarks1 (numpy.ndarray): 첫 번째 얼굴의 랜드마크 배열.
        landmarks2 (numpy.ndarray): 두 번째 얼굴의 랜드마크 배열.
        max_diff (float): 최대 허용 비율 차이 (기본값: 0.2).

    Returns:
        float: 0~1 범위의 유사도 점수.
    """
    return calculate_position_ratio_similarity(landmarks1, landmarks2, position_index=30, max_diff=max_diff)


def calculate_mouth_position_ratio_similarity(landmarks1, landmarks2, max_diff=0.2):
    """
    두 얼굴의 입 위치 비율 유사도를 계산합니다.

    Args:
        landmarks1 (numpy.ndarray): 첫 번째 얼굴의 랜드마크 배열.
        landmarks2 (numpy.ndarray): 두 번째 얼굴의 랜드마크 배열.
        max_diff (float): 최대 허용 비율 차이 (기본값: 0.2).

    Returns:
        float: 0~1 범위의 유사도 점수.
    """
    return calculate_position_ratio_similarity(landmarks1, landmarks2, position_index=51, max_diff=max_diff)


# 유사도 계산 함수
def calculate_similarity(father_img, mother_img, child_img, sibling_files):
    if not (father_img and mother_img and child_img):
        return "모든 필수 이미지를 업로드하세요."

    sibling_images = [Image.open(f) for f in sibling_files] if sibling_files else []

    # 얼굴 인식 상태 확인
    messages, success = update_recognition_status(father_img, mother_img, child_img, sibling_files)
    if not success:
        return f"{messages}\n\n유사도 측정을 할 수 없습니다."

    try:
        # Dlib 랜드마크 감지를 위한 초기화
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor("/content/drive/My Drive/shape_predictor_68_face_landmarks.dat")

        def get_landmarks(img, role):
            """이미지에서 랜드마크를 감지합니다."""
            img_array = np.array(img.convert("RGB"))
            gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
            faces = detector(gray)
            if len(faces) == 0:
                return None
            landmarks = predictor(gray, faces[0])
            return landmarks_to_numpy(landmarks)

        # 랜드마크 저장
        landmarks_detected = {
            "Father": get_landmarks(father_img, "Father"),
            "Mother": get_landmarks(mother_img, "Mother"),
            "Child": get_landmarks(child_img, "Child"),
        }
        for i, sibling_img in enumerate(sibling_images):
            landmarks_detected[f"Sibling {i+1}"] = get_landmarks(sibling_img, f"Sibling {i+1}")

        # DeepFace 기반 임베딩 생성
        def get_embedding(img):
            img_array = np.array(img.convert("RGB"))
            if img_array.size == 0:
                raise ValueError("이미지가 비어 있습니다.")
            return DeepFace.represent(img_path=img_array, model_name="VGG-Face", enforce_detection=False)[0]["embedding"]

        embeddings = {
            "Father": get_embedding(father_img),
            "Mother": get_embedding(mother_img),
            "Child": get_embedding(child_img),
        }
        for i, sibling_img in enumerate(sibling_images):
            embeddings[f"Sibling {i+1}"] = get_embedding(sibling_img)

        # 메트릭 기반 유사도 계산
        similarities = {}

        def calculate_similarities(role1, role2, img1, img2):
            """두 역할(role1, role2) 간 유사도를 계산합니다."""
            img1_array = np.array(img1.convert("RGB"))
            img2_array = np.array(img2.convert("RGB"))

            if landmarks_detected[role1] is not None and landmarks_detected[role2] is not None:
                lip_similarity = calculate_lip_curvature_similarity(
                    landmarks_detected[role1], landmarks_detected[role2]
                )
                eye_pos_similarity = calculate_eye_position_ratio_similarity(
                    landmarks_detected[role1], landmarks_detected[role2]
                )
                nose_pos_similarity = calculate_nose_position_ratio_similarity(
                    landmarks_detected[role1], landmarks_detected[role2]
                )
                mouth_pos_similarity = calculate_mouth_position_ratio_similarity(
                    landmarks_detected[role1], landmarks_detected[role2]
                )
            else:
                lip_similarity = eye_pos_similarity = nose_pos_similarity = mouth_pos_similarity = 0

            similarities[f"{role1}-{role2}"] = {
                "feature_distribution": calculate_cosine_similarity(
                    embeddings[role1], embeddings[role2]
                ),
                "eye_color": calculate_rgb_similarity(
                    img1_array, img2_array
                ),
                "nose_shape": calculate_ssim_similarity(
                    img1_array, img2_array, target_size=(224, 224)
                ),
                "mouth_shape": lip_similarity,
                "eye_position_ratio": eye_pos_similarity,
                "nose_position_ratio": nose_pos_similarity,
                "mouth_position_ratio": mouth_pos_similarity,
            }

        # 관계별 유사도 계산
        calculate_similarities("Father", "Child", father_img, child_img)
        calculate_similarities("Mother", "Child", mother_img, child_img)
        for i, sibling_img in enumerate(sibling_images):
            calculate_similarities(f"Sibling {i+1}", "Child", sibling_img, child_img)

        # 가중치 기반 최종 유사도 계산
        def calculate_weighted_similarity(metrics):
            return sum(weights[key] * value for key, value in metrics.items() if key in weights)

        # 유사도 결과 메시지 생성
        def generate_similarity_message(similarity_score):
            percentage = similarity_score * 100
            if percentage > 60:
                return "동일인 아닌가요?"
            elif 50 <= percentage <= 60:
                return "붕어빵입니다"
            elif 40 <= percentage < 50:
                return "아주 닮았어요"
            elif 30 <= percentage < 40:
                return "닮았어요"
            elif 20 <= percentage < 30:
                return "잘 모르겠는데요?"
            elif 10 <= percentage < 20:
                return "딱히 안 닮았는데요?"
            else:
                return "유전자 검사 해봐요!"

        # 최종 결과
        results = []
        for role, img in [("Father", father_img), ("Mother", mother_img)] + [
            (f"Sibling {i+1}", sibling_img) for i, sibling_img in enumerate(sibling_images)
        ]:
            role_similarity = calculate_weighted_similarity(similarities[f"{role}-Child"])
            similarity_message = generate_similarity_message(role_similarity)
            results.append(f"{role}: {role_similarity * 100:.2f}% 닮았습니다. {similarity_message}")

        return "\n".join(results)

    except Exception as e:
        return f"유사도 계산 중 오류 발생: {str(e)}"

# Gradio UI 구성
title = "가족 유사도 분석"
description = "사진을 업로드하면 얼굴 인식 결과가 즉시 표시됩니다. 유사도 분석 버튼을 눌러 결과를 확인하세요."

with gr.Blocks() as interface:
    gr.Markdown(f"# {title}")
    gr.Markdown(description)

    with gr.Row():
        father_img = gr.Image(label="Father 사진 업로드", type="pil")
        mother_img = gr.Image(label="Mother 사진 업로드", type="pil")
        child_img = gr.Image(label="Child 사진 업로드", type="pil")

    sibling_upload = gr.File(label="Sibling 사진 업로드 (선택)", file_types=[".jpg", ".png"], file_count="multiple")
    sibling_gallery = gr.Gallery(label="Sibling 사진 미리보기", show_label=True, elem_id="sibling-gallery")
    recognition_status = gr.Textbox(label="얼굴 인식 결과", interactive=False)
    analyze_button = gr.Button("유사도 분석")
    similarity_results = gr.Textbox(label="유사도 분석 결과", interactive=False)

    # Sibling 미리보기 업데이트 함수
    def update_sibling_gallery(sibling_files):
        if sibling_files:
            return [Image.open(f) for f in sibling_files]
        return []

    # 즉시 업데이트: 얼굴 인식 상태
    def immediate_check_and_analyze(father_img, mother_img, child_img, sibling_files):
        messages, _ = update_recognition_status(father_img, mother_img, child_img, sibling_files)
        return messages

    inputs = [father_img, mother_img, child_img, sibling_upload]
    father_img.change(fn=immediate_check_and_analyze, inputs=inputs, outputs=recognition_status)
    mother_img.change(fn=immediate_check_and_analyze, inputs=inputs, outputs=recognition_status)
    child_img.change(fn=immediate_check_and_analyze, inputs=inputs, outputs=recognition_status)
    sibling_upload.upload(fn=immediate_check_and_analyze, inputs=inputs, outputs=recognition_status)
    sibling_upload.upload(fn=update_sibling_gallery, inputs=sibling_upload, outputs=sibling_gallery)

    analyze_button.click(fn=calculate_similarity, inputs=inputs, outputs=similarity_results)

interface.launch(share=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d727419dd07b942d67.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
